In [1]:
import os

os.chdir("..\\..\\tensorflow")

In [2]:
import subprocess

def run_cmd(cmd):
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    while True:
        output = proc.stdout.readline()
        if proc.poll() is not None:
            break
        if output:
            try:
                print(output.decode('utf-8').replace('\n', ''))
            except UnicodeDecodeError:
                pass
    rc = proc.poll()

# Deploy to SparkFun Edge  
The following instructions will help you build and deploy this example on the SparkFun Edge development board.

The program will toggle the blue LED on and off with each inference. It will switch on the yellow LED when a "yes" is heard, the red LED when a "no" is heard, and the green LED when an unknown command is heard.

The AI on a microcontroller with TensorFlow Lite and SparkFun Edge walks through the deployment process in detail. The steps are also summarized below.

## Compile the binary  
The following command will download the required dependencies and then compile a binary for the SparkFun Edge:

In [3]:
cmd = 'bash -c "make -f tensorflow/lite/micro/tools/make/Makefile TARGET=sparkfun_edge TAGS=\"cmsis-nn\" micro_speech_bin"'
run_cmd(cmd)

downloading http://github.com/ARM-software/CMSIS_5/archive/01f5b32badf7b78c85a24a7149b56400fa6a2999.zip
tensorflow/lite/micro/tools/make/Makefile:417: warning: overriding recipe for target 'tensorflow/lite/micro/tools/make/downloads/person_model_int8'
tensorflow/lite/micro/tools/make/Makefile:417: warning: ignoring old recipe for target 'tensorflow/lite/micro/tools/make/downloads/person_model_int8'
tensorflow/lite/micro/tools/make/download_and_extract.sh "https://github.com/google/gemmlowp/archive/719139ce755a0f31cbf1c37f7f98adcc7fc9f425.zip" "7e8191b24853d75de2af87622ad293ba" tensorflow/lite/micro/tools/make/downloads/gemmlowp  
downloading https://github.com/google/gemmlowp/archive/719139ce755a0f31cbf1c37f7f98adcc7fc9f425.zip
tensorflow/lite/micro/tools/make/download_and_extract.sh "http://mirror.tensorflow.org/github.com/google/flatbuffers/archive/dca12522a9f9e37f126ab925fd385c807ab4f84e.zip" "aa9adc93eb9b33fa1a2a90969e48baee" tensorflow/lite/micro/tools/make/downloads/flatbuffers  

The binary will be created in the following location:

In [4]:
cmd = 'bash -c "ls -l tensorflow/lite/micro/tools/make/gen/sparkfun_edge_cortex-m4/bin/micro_speech.bin"'
run_cmd(cmd)

-rw-r--r-- 1 aaron aaron 116884 Nov 25 23:14 tensorflow/lite/micro/tools/make/gen/sparkfun_edge_cortex-m4/bin/micro_speech.bin


## Sign the binary  
The binary must be signed with cryptographic keys to be deployed to the device. We'll now run some commands that will sign our binary so it can be flashed to the SparkFun Edge. The scripts we are using come from the Ambiq SDK, which is downloaded when the Makefile is run.  
  
Enter the following command to set up some dummy cryptographic keys we can use for development:

In [5]:
cmd = 'bash -c "cp tensorflow/lite/micro/tools/make/downloads/AmbiqSuite-Rel2.2.0/tools/apollo3_scripts/keys_info0.py tensorflow/lite/micro/tools/make/downloads/AmbiqSuite-Rel2.2.0/tools/apollo3_scripts/keys_info.py"'
run_cmd(cmd)

Next, run the following command to create a signed binary:

In [6]:
cmd = 'python tensorflow/lite/micro/tools/make/downloads/AmbiqSuite-Rel2.2.0/tools/apollo3_scripts/create_cust_image_blob.py --bin tensorflow/lite/micro/tools/make/gen/sparkfun_edge_cortex-m4/bin/micro_speech.bin --load-address 0xC000 --magic-num 0xCB -o main_nonsecure_ota --version 0x0'
run_cmd(cmd)

Header Size =  0x80
original app_size  0x1c894 ( 116884 )
load_address  0xc000 ( 49152 )
app_size  0x1c894 ( 116884 )
w0 = 0xcb01c914
Security Value  0x10
w2 =  0x10008080
addrWord =  0xc000
versionKeyWord =  0x0
child0/feature =  0xffffffff
child1 =  0xffffffff
crc =   0x818c74f3
Writing to file  main_nonsecure_ota.bin


This will create the file main_nonsecure_ota.bin. We'll now run another command to create a final version of the file that can be used to flash our device with the bootloader script we will use in the next step:

In [7]:
cmd = 'python tensorflow/lite/micro/tools/make/downloads/AmbiqSuite-Rel2.2.0/tools/apollo3_scripts/create_cust_wireupdate_blob.py --load-address 0x20000 --bin main_nonsecure_ota.bin -i 6 -o main_nonsecure_wire --options 0x1'
run_cmd(cmd)

Header Size =  0x60
app_size  0x1c914 ( 117012 )
Writing to file  main_nonsecure_wire.bin
Image from  0x0  to  0x1c914  will be loaded at 0x20000


You should now have a file called main_nonsecure_wire.bin in the directory where you ran the commands. This is the file we'll be flashing to the device.

## Flash the binary  
Next, attach the board to your computer via a USB-to-serial adapter.  

Note: If you're using the SparkFun Serial Basic Breakout, you should install the latest drivers before you continue.  

Once connected, assign the USB device name to an environment variable:

In [8]:
DEVICENAME='COM3'

Set another variable with the baud rate:

In [9]:
BAUD_RATE='921600'

Now, hold the button marked 14 on the device. While still holding the button, hit the button marked RST. Continue holding the button marked 14 while running the following command:

In [10]:
cmd = 'python tensorflow/lite/micro/tools/make/downloads/AmbiqSuite-Rel2.2.0/tools/apollo3_scripts/uart_wired_update.py -b ' + BAUD_RATE + ' ' + DEVICENAME + ' -r 1 -f main_nonsecure_wire.bin -i 6'
run_cmd(cmd)

Connecting with Corvette over serial port COM3...
Sending Hello.
Received response for Hello
Received Status
length =  0x58
version =  0x3
Max Storage =  0x4ffa0
Status =  0x2
State =  0x7
AMInfo = 
0x1
0xff2da3ff
0x55fff
0x1
0x49f40003
0xffffffff
0xffffffff
0xffffffff
0xffffffff
0xffffffff
0xffffffff
0xffffffff
0xffffffff
0xffffffff
0xffffffff
0xffffffff
Sending OTA Descriptor =  0xfe000
Sending Update Command.
number of updates needed =  1
Sending block of size  0x1c974  from  0x0  to  0x1c974
Sending Data Packet of length  8180
Sending Data Packet of length  8180
Sending Data Packet of length  8180
Sending Data Packet of length  8180
Sending Data Packet of length  8180
Sending Data Packet of length  8180
Sending Data Packet of length  8180
Sending Data Packet of length  8180
Sending Data Packet of length  8180
Sending Data Packet of length  8180
Sending Data Packet of length  8180
Sending Data Packet of length  8180
Sending Data Packet of length  8180
Sending Data Packet of length  